# Check the running time of ROCKET

In [1]:
from rocket._kernels import _generate_1d_kernels, _apply_1d_kernel, _generate_nd_kernels, _apply_nd_kernel, _apply_1d_kernels, _apply_nd_kernels
from rocket import Rocket
from dataloader.dataset import UniformSegmentDataset
from param import *

In [2]:
data_dir = ParamDir().data_path_list[0]
dataset = UniformSegmentDataset(data_dir, ParamData().mobility, ParamData().shuffle)
(X_train, y_train), (X_test, y_test) = dataset.load_all_data(ParamData().window_size, ParamData().train_ratio, ParamData().K)

## rocket.fit_transform

In [7]:
%%timeit
rocket = Rocket(100, 2, 20230405)
X_transformed = rocket.fit_transform(X_train)

18.7 s ± 798 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Optimization

### 1. _generate_nd_kernels

In [3]:
%%timeit
# before njit
kernels = _generate_nd_kernels(50, 16, 100, 2, 20230404)

25.7 ms ± 248 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### 1.1 numba safe way of combinations

In [3]:
from util import get_random_comb

In [5]:
%%timeit
# with njit
get_random_comb(np.arange(0, 50),3, 3, 2)

9.67 µs ± 657 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### 1.2 results with njit

In [3]:
%%timeit
# with njit
kernels = _generate_nd_kernels(50, 16, 100, 2, 20230404)

1.34 ms ± 583 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2. _apply_nd_kernel

In [3]:
%%timeit
kernels = _generate_nd_kernels(50, 16, 100, 2, 20230404)
kernel_dim = 2
(
    weights,
    length_list,
    biases,
    dilations,
    paddings,
    num_combinations_list,
    combinations_indices,
) = kernels

s_w, s_f, s_out_f = 0, 0, 0
j = 0
e_w = s_w + num_combinations_list[j] * length_list[j] * kernel_dim # end index of weights
e_f = s_f + num_combinations_list[j] # end index of features to use
_weights = weights[s_w:e_w].reshape((num_combinations_list[j], length_list[j], kernel_dim))
kernel = (
            _weights,
            length_list[j],
            biases[j],
            dilations[j],
            paddings[j],
            num_combinations_list[j],
            combinations_indices[s_f:e_f]
            )

features = _apply_nd_kernel(X_train[0], kernel, kernel_dim)

2.58 ms ± 715 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 2.1 results after njit

In [3]:
%%timeit

kernels = _generate_nd_kernels(50, 16, 100, 2, 20230404)
kernel_dim = 2
(
    weights,
    length_list,
    biases,
    dilations,
    paddings,
    num_combinations_list,
    combinations_indices,
) = kernels

s_w, s_f, s_out_f = 0, 0, 0
j = 0
e_w = s_w + num_combinations_list[j] * length_list[j] * kernel_dim # end index of weights
e_f = s_f + num_combinations_list[j] # end index of features to use
_weights = weights[s_w:e_w].reshape((num_combinations_list[j], length_list[j], kernel_dim))
kernel = (
            _weights,
            length_list[j],
            biases[j],
            dilations[j],
            paddings[j],
            num_combinations_list[j],
            combinations_indices[s_f:e_f]
            )
_apply_nd_kernel(X_train[0], kernel, kernel_dim)

1.36 ms ± 670 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3. _generate_1d_kernels

In [10]:
%%timeit
# before njit
kernels = _generate_1d_kernels(50, 16, 100, 2)

19.9 ms ± 377 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### 3.1 results after njit

In [3]:
%%timeit
# before njit
kernels = _generate_1d_kernels(50, 16, 100, 2)

653 µs ± 429 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 4. _apply_1d_kernel

In [4]:
%%timeit
kernels = _generate_1d_kernels(50, 16, 100, 2)

(
    weights,
    length_list,
    biases,
    dilations,
    paddings,
    num_features_list,
    features_indices,
) = kernels

s_w, s_f, s_out_f = 0, 0, 0
j = 0
e_w = s_w + num_features_list[j] * length_list[j]  # end index of weights
e_f = s_f + num_features_list[j] # end index of features to use
_weights = weights[s_w:e_w].reshape((num_features_list[j], length_list[j]))
kernel = (
            _weights,
            length_list[j],
            biases[j],
            dilations[j],
            paddings[j],
            num_features_list[j],
            features_indices[s_f:e_f]
            )

features = _apply_1d_kernel(X_train[0], kernel)

1.39 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 4.1 results after njit

In [5]:
%%timeit
kernels = _generate_1d_kernels(50, 16, 100, 2)

(
    weights,
    length_list,
    biases,
    dilations,
    paddings,
    num_features_list,
    features_indices,
) = kernels

s_w, s_f, s_out_f = 0, 0, 0
j = 0
e_w = s_w + num_features_list[j] * length_list[j]  # end index of weights
e_f = s_f + num_features_list[j] # end index of features to use
_weights = weights[s_w:e_w].reshape((num_features_list[j], length_list[j]))
kernel = (
            _weights,
            length_list[j],
            biases[j],
            dilations[j],
            paddings[j],
            num_features_list[j],
            features_indices[s_f:e_f]
            )

features = _apply_1d_kernel(X_train[0], kernel)

466 µs ± 4.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 5. _apply_1d_kernels

In [5]:
%%timeit
# for 1d kernels
kernels = _generate_1d_kernels(50, 16, 100, 2)

_apply_1d_kernels(X_train, kernels)

144 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### 5.1 results after njit

In [3]:
%%timeit
# for 1d kernels
kernels = _generate_1d_kernels(50, 16, 100, 2)

_apply_1d_kernels(X_train, kernels)

21.8 ms ± 475 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 5.2 results after parallel

In [6]:
%%timeit
# for 1d kernels
nb.set_num_threads(30)
kernels = _generate_1d_kernels(50, 16, 100, 2)
_apply_1d_kernels(X_train, kernels)

3.42 ms ± 421 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 6. _apply_nd_kernels

In [3]:
%%timeit
kernels = _generate_nd_kernels(50, 16, 100, 2, 20230405)
kernel_dim = 2
_apply_nd_kernels(X_train, kernels, kernel_dim)

207 ms ± 2.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 6.1 results after njit

In [4]:
%%timeit
kernels = _generate_nd_kernels(50, 16, 100, 2, 20230405)
kernel_dim = 2
_apply_nd_kernels(X_train, kernels, kernel_dim)

37.7 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### 6.2 results after paral

In [3]:
%%timeit
# for 1d kernels
nb.set_num_threads(30)
kernels = _generate_nd_kernels(50, 16, 100, 2, 20230405)
kernel_dim = 2
_apply_nd_kernels(X_train, kernels, kernel_dim)

7 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Results after optimization

In [3]:
%%timeit
rocket = Rocket(100, 2, 20230405, 30)
X_transformed = rocket.fit_transform(X_train)

5.17 ms ± 1.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
